### Transformación de la base de datos para poderla trabajar


In [19]:
import pandas as pd
import numpy
import locale
from dateutil.parser import parse

In [20]:
# Cargamos el csv con los datos finales
datos = pandas.read_csv("Final_data.csv")

datos['Date'] = datos['Date'].apply(lambda x: x.replace('abri', 'abril'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('gosto', 'agosto'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('dciembre', 'diciembre'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('dicienbre', 'diciembre'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('abrill', 'abril'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('aagosto', 'agosto'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('Aagosto', 'agosto'))

# eliminar el día de la semana de las cadenas de fecha
datos['Date_2'] = datos['Date'].str.replace(r'^\w+\s+', '')

# convertir la columna de fecha en datetime
datos['Date_2'] = pandas.to_datetime(datos['Date_2'], format='%d de %B de %Y', errors='coerce')


fechas = datos['Date'].unique()



C:\Users\Jorge De León\AppData\Local\Temp\ipykernel_5084\2568158253.py:2: DtypeWarning: Columns (25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,42,43,44,45,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pandas.read_csv("Final_data.csv")
C:\Users\Jorge De León\AppData\Local\Temp\ipykernel_5084\2568158253.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  datos['Date_2'] = datos['Date'].str.replace(r'^\w+\s+', '')


In [21]:
locale.setlocale(locale.LC_TIME, 'es_CO.UTF-8')

#datos['Date-2'] = pandas.to_datetime(datos['Date'], format='%A %d de %B de %Y', errors='coerce')

fechas = pandas.DataFrame(fechas)
fechas['date'] = fechas[0].str.replace(r'^\w+\s+', '')

# convertir la columna de fecha en datetime
fechas['date'] = pandas.to_datetime(fechas['date'], format='%d de %B de %Y', errors='coerce')

fechas_malas_2 = fechas.loc[fechas['date'].isna()]


C:\Users\Jorge De León\AppData\Local\Temp\ipykernel_5084\1537350283.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  fechas['date'] = fechas[0].str.replace(r'^\w+\s+', '')


In [22]:
datos_finale = datos.drop(['Date'], axis=1)

In [23]:

# Define the time range for the dates
start_date = datos_finale['Date_2'].min()
end_date = datos_finale['Date_2'].max()
date_range = pd.date_range(start_date, end_date, freq='D')

# Create a DataFrame with all unique categories in column 'A'
categories = datos_finale['Precio $/Kg'].unique()
categories_df = pd.DataFrame({'Precio $/Kg': categories})

# Cross join the categories DataFrame with the date range
categories_df['key'] = 1
date_range_df = pd.DataFrame({'key': 1, 'Date_2': date_range})
complete_df = categories_df.merge(date_range_df, on='key').drop('key', axis=1)

# Merge the complete DataFrame with the original DataFrame
df_complete = complete_df.merge(datos_finale, on=['Precio $/Kg', 'Date_2'], how='left')

# Sort the DataFrame by category and date
df_complete = df_complete.sort_values(['Precio $/Kg', 'Date_2'])

# Reset the index to default
df_complete = df_complete.reset_index(drop=True)

columns_to_fill = datos_finale.columns.difference(['Precio $/Kg', 'Date_2'])
df_complete[columns_to_fill] = df_complete.groupby('Precio $/Kg')[columns_to_fill].fillna(value=pd.NA)

In [24]:
FINAL = df_complete
FINAL.to_csv("Final_database_2.csv")